# Исследование надежности заемщиков

В проекте нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Заказчик — кредитный отдел банка. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Загрузка таблицы и изучение общей информации о данных

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("C:/Users/Turner/Documents/datasets/credits.csv")
display(data.head(20))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


## Предобработка данных

#### Выведем количество пропущенных значений для каждого столбца.

In [4]:
print(data.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


Первый столбец с пропущенными значениями — total_income — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости. 

#### Поэтому заполним пропуски в этом столбце медианным значением по каждому типу из столбца income_type.

In [5]:
medians = data.groupby('income_type')['total_income'].transform('median')
data['total_income']=data['total_income'].fillna(medians)

In [6]:
print(data['days_employed'].min())

-18388.949900568383


Наблюдается отрицательное количество дней трудового стажа в столбце days_employed. 

#### Уберём его, применив функцию abs.

In [7]:
data['days_employed']=abs(data['days_employed'])

#### Для каждого типа занятости выведем медианное значение трудового стажа days_employed в днях.

In [8]:
display(data.groupby('income_type', as_index = False)['days_employed'].median())

,income_type,days_employed
0,безработный,366413.652744
1,в декрете,3296.759962
2,госслужащий,2689.368353
3,компаньон,1547.382223
4,пенсионер,365213.306266
5,предприниматель,520.848083
6,сотрудник,1574.202821
7,студент,578.751554


У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть.

#### Выведем перечень уникальных значений столбца children.

In [9]:
print(data['children'].unique())

[ 1  0  3  2 -1  4 20  5]


#### В столбце children есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма data.

In [10]:
data = data[data['children'] != -1 | 20]
data = data[data['children'] != 20]
print(data['children'].unique())

[1 0 3 2 4 5]


#### Заполним пропуски в столбце `days_employed` медианными значениями по каждому типу занятости `income_type`.

In [11]:
medians2 = data.groupby('income_type')['days_employed'].transform('median')
data['days_employed']=data['days_employed'].fillna(medians2)

#### Проверим выводом количества пропущенных значений для каждого столбца.

In [12]:
print(data.isna().sum())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


#### Заменим вещественный тип данных в столбце `total_income` на целочисленный.

In [13]:
data['total_income'] = data['total_income'].astype('int')

#### Обработаем неявные дубликаты в столбце education. 

В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведём их к нижнему регистру. 

In [14]:
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
print(data['education'].unique())
for row in data.columns:
    print(data[row].unique())

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']
[1 0 3 2 4 5]
[8437.67302776 4024.80375385 5623.42261023 ... 2113.3468877  3112.4817052
 1984.50758853]
[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]
['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']
[0 1 2 3 4]
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'не женат / не замужем']
[0 1 2 3 4]
['F' 'M' 'XNA']
['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
[0 1]
[253875 112080 145885 ...  89672 244093  82047]
['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство

#### Посмотрим есть ли дубликаты и при наличии избавимся от них.

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

На основании диапазонов, указанных ниже, создадим в датафрейме data столбец total_income_category с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [17]:
# создаём функцию categorize_income()
def categorize_income(money):
    if money <= 30000:
        return 'E'
    if money <= 50000:
        return 'D'
    if money <= 200000:
        return 'C'
    if money <= 1000000:
        return 'B'
    else:
        return 'A'
    
data['total_income_category'] = (data['total_income']).apply(categorize_income)

#### Выведем на экран перечень уникальных целей взятия кредита из столбца purpose.

In [18]:
print(data['purpose'].unique())

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [19]:
def categorize_purpose(purpose):
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'образ' in purpose:
        return 'получение образования'
    if 'жил' or 'недвиж' in purpose:
        return 'операции с недвижимостью'
    
# применим функцию методом apply()
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Исследование данных

#### Выясним, есть ли зависимость между количеством детей и возвратом кредита в срок. Сгруппируем данные по количеству детей и просуммируем данные из столбца 'долг', который равен единице, если была просрочка по платежу.

In [20]:
display(data.groupby('children', as_index = False)['debt'].sum())

,children,debt
0,0,1063
1,1,444
2,2,194
3,3,27
4,4,4
5,5,0


#### Разделим количество должников на количество клиентов, относящихся к каждой группе.

In [21]:
display((data.groupby('children')['debt'].sum()/
        data.groupby('children')['debt'].count()).
        reset_index())

,children,debt
0,0,0.075438
1,1,0.092346
2,2,0.094542
3,3,0.081818
4,4,0.097561
5,5,0.000000


Выходит, что количество детей практически не влияет на способность возвращать долги в срок. Разве что бездетным немного проще, что логично. И, конечно, бросается в глаза аномалия - провал, где 3 ребёнка. Клиенты с 5 детьми не допускают просрочек, но и таковых в принципе мало. Однако репрезентативный вывод по заёмщикам с 3-5 детьми сделать нельзя, т.к. эти категории составляют менее 5 % от всего количества срок в датафрейме.

#### Выяним, есть ли зависимость между семейным положением и возвратом кредита в срок. Аналогично сгруппируем данные по стобцу "семейное положение" и просуммируем данные из стобца 'долг'. Затем выведем процентные соотношения.

In [22]:
display((data.groupby('family_status')['debt'].sum()/
        data.groupby('family_status')['debt'].count()).
        reset_index())

,family_status,debt
0,в разводе,0.070648
1,вдовец / вдова,0.066246
2,гражданский брак,0.093130
3,женат / замужем,0.075606
4,не женат / не замужем,0.097639


Больше всего просрочек у официально безбрачных клиентов. У остальных несколько меньше и примерно одинаково. Меньше всего у вдов и вдовцов. Клиенты, когда-либо вступавшие в брак, надёжнее.

#### Выясним, есть ли зависимость между уровнем дохода и возвратом кредита в срок.

Поскольку уровень дохода является количественной переменной, придётся исследовать диапазоны значений. Чтобы корректнее 
их определить, выведем максимальное, минимальное значения, медиану и среднее по стобцу "уровень дохода".

In [23]:
display(data['total_income'].describe().reset_index())

,index,total_income
0,count,2.133100e+04
1,mean,1.653427e+05
2,std,9.831399e+04
3,min,2.066700e+04
4,25%,1.075070e+05
5,50%,1.425940e+05
6,75%,1.958420e+05
7,max,2.265604e+06


#### Отсортируем значения уровня дохода на 4 диапазона: 20667-100000, 100001-142000, 142000-200000, 200001-2265604 и затем выведем отношения просуммированных просрочек к кол-ву клиентов для каждого диапазона количество просрочек.

In [24]:
print('Клиенты:')
print('Диапазон 20667-100000 :', data.loc[data['total_income'] <= pow(10,5), 'debt'].count())
print('Диапазон 100001-142000 :', data.loc[(data['total_income'] <= 1.42*pow(10,5)) & (data['total_income'] > pow(10,5)), 'debt'].count())
print('Диапазон 142001-200000 :', data.loc[(data['total_income'] <= 2*pow(10,5)) & (data['total_income'] > 1.42*pow(10,5)), 'debt'].count())
print('Диапазон 200001-2265604 :', data.loc[data['total_income'] > 2*pow(10,5), 'debt'].count())

value_1 = round((data.loc[data['total_income'] <= pow(10,5), 'debt'].sum() /
     data.loc[data['total_income'] <= pow(10,5), 'debt'].count()), 3)
value_2 = round((data.loc[(data['total_income'] <= 1.42*pow(10,5)) & (data['total_income'] > pow(10,5)), 'debt'].sum() / 
     data.loc[(data['total_income'] <= 1.42*pow(10,5)) & (data['total_income'] > pow(10,5)), 'debt'].count()), 3)
value_3 = round((data.loc[(data['total_income'] <= 2*pow(10,5)) & (data['total_income'] > 1.42*pow(10,5)), 'debt'].sum() / 
     data.loc[(data['total_income'] <= 2*pow(10,5)) & (data['total_income'] > 1.42*pow(10,5)), 'debt'].count()), 3)
value_4 = round((data.loc[data['total_income'] > 2*pow(10,5), 'debt'].sum() / 
     data.loc[data['total_income'] > 2*pow(10,5), 'debt'].count()), 3)
print('')
print('Доля должников:')
print('Диапазон 20667-100000 :', value_1)
print('Диапазон 100001-142000 :', value_2)
print('Диапазон 142001-200000 :', value_3)
print('Диапазон 200001-500000 :', value_4)

Клиенты:
Диапазон 20667-100000 : 4444
Диапазон 100001-142000 : 5199
Диапазон 142001-200000 : 6649
Диапазон 200001-2265604 : 5039

Доля должников:
Диапазон 20667-100000 : 0.08
Диапазон 100001-142000 : 0.086
Диапазон 142001-200000 : 0.087
Диапазон 200001-500000 : 0.071


Получается, что клиенты с малым и средним доходом примерно в равных количествах допускают просрочки по платежам. С ростом дохода способность выплачивать долги в срок понемногу растёт. Кол-во клиентов четырёх групп имеют один порядок, значит выводы по ним количественно подкреплены.

#### Как разные цели кредита влияют на его возврат в срок

Аналогично предыдущим пунктам сгруппируем данные по стобцу "цели" и выведем отношения просуммированных просрочек к кол-ву клиентов для каждого диапазона количество просрочек.

In [25]:
display((data.groupby('purpose_category')['debt'].sum() / 
        data.groupby('purpose_category')['debt'].count()).
        reset_index())

,purpose_category,debt
0,операции с автомобилем,0.093480
1,операции с недвижимостью,0.072551
2,получение образования,0.092528
3,проведение свадьбы,0.079118


На авто и образование приходится больше просрочек, чем на устройство свадеб и покупку недвижимости.

#### Приведём возможные причины появления пропусков в исходных данных.

1. Клиент не предоставил документы о своём трудовом стаже.
2. Менеджер не заполнил все поля при наличии документов клиента.
2. Ошибка при выгрузке базы данных.
3. Данные о доходах отдельных клиентов, возможно, скрыты намеренно.

### Выводы.

#### 1. Бездетные допускают меньше просрочек, чем клиенты с детьми. Клиенты с 5 детьми не допускают просрочек. В остальных случаях количество детей практически не влияет на способность возвращать долги в срок. Несколько выше способность возращать в срок долги у клиентов с 3 детьми. Предполагаю, что это аномальное значение.

#### 2. Больше всего просрочек у официально безбрачных клиентов. У остальных несколько меньше и примерно одинаково. Меньше всего просрочек у вдов и вдовцов.

#### 3. Клиенты с малым и средним доходом примерно в равных количествах допускают просрочки по платежам. С ростом дохода способность выплачивать долги в срок понемногу растёт.

#### 4. На покупку авто и получение образования приходится больше просрочек, чем на устройство свадеб и покупку недвижимости.